In [ ]:
import os
import cv2
import matplotlib.pyplot as plt

train_directory = '/kaggle/input/testbens/testbens'
class_directories = ['0', '1']  # Sub-directory names for each class
IMG_SIZE = (224, 224)  # Desired image size for display

# Define the pre-processing functions
def crop_image_from_gray(img, tol=7):
    if img.ndim == 2:
        mask = img > tol
        return img[np.ix_(mask.any(1), mask.any(0))]
    elif img.ndim == 3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img > tol

        check_shape = img[:, :, 0][np.ix_(mask.any(1), mask.any(0))].shape[0]
        if check_shape == 0:
            return img
        else:
            img1 = img[:, :, 0][np.ix_(mask.any(1), mask.any(0))]
            img2 = img[:, :, 1][np.ix_(mask.any(1), mask.any(0))]
            img3 = img[:, :, 2][np.ix_(mask.any(1), mask.any(0))]
            img = np.stack([img1, img2, img3], axis=-1)
        return img

def preprocess_image(path, sigmaX=10):
    image = cv2.imread(path)
    if image is None:
        return None

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = crop_image_from_gray(image)
    image = cv2.resize(image, IMG_SIZE)
    image = cv2.addWeighted(image, 4, cv2.GaussianBlur(image, (0, 0), sigmaX), -4, 128)

    return image

# Display original and pre-processed images
fig, axes = plt.subplots(nrows=len(class_directories), ncols=6, figsize=(15, 10))

for i, class_dir in enumerate(class_directories):
    image_files = os.listdir(os.path.join(train_directory, class_dir))
    image_files = image_files[:3]  # Select first 3 images from each class

    for j, image_file in enumerate(image_files):
        original_image_path = os.path.join(train_directory, class_dir, image_file)
        preprocessed_image = preprocess_image(original_image_path)

        # Display original image
        original_image = cv2.imread(original_image_path)
        original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
        axes[i, j * 2].imshow(original_image)
        axes[i, j * 2].set_title(class_dir)
        axes[i, j * 2].axis('off')

        # Display pre-processed image
        axes[i, j * 2 + 1].imshow(preprocessed_image)
        if class_dir == '0':
            axes[i, j * 2 + 1].set_title("Non-Referable")
        elif class_dir == '1':
            axes[i, j * 2 + 1].set_title("Referable")
        axes[i, j * 2 + 1].axis('off')

plt.tight_layout()
plt.show()
